In [22]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
from requests_testadapter import Resp
import requests
#from requests_file import FileAdapter
import os.path
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
import nltk

In [23]:
df_reviews = pd.read_csv('reviews_sep2_clean.csv')

In [24]:
print ('valores unicos en COMIDA',df_reviews.comida.unique())
print ('valores unicos en SERVICIO',df_reviews.servicio.unique())
print ('valores unicos en AMBIENTE',df_reviews.ambiente.unique())
df_reviews.head(3)

valores unicos en COMIDA ['Excelente' 'Muy bueno' 'Bueno' 'Regular']
valores unicos en SERVICIO ['Excelente' 'Regular' 'Muy bueno' 'Bueno']
valores unicos en AMBIENTE ['Excelente' 'Muy bueno' 'Regular' 'Bueno']


,resto,fecha,review,comida,servicio,ambiente,tokened,normal,nonstop,nopoint
0,-amigos-del-mar-608,12/07/2015,el mejor lugar de la Argentina para comer comi...,Excelente,Excelente,Excelente,"['el', 'mejor', 'lugar', 'de', 'la', 'Argentin...","['el', 'mejor', 'lugar', 'de', 'la', 'argentin...","['mejor', 'lugar', 'argentina', 'comer', 'comi...","['mejor', 'lugar', 'argentina', 'comer', 'comi..."
1,-amigos-del-mar-608,22/06/2015,"El Curry Udon excelente, hacía mucho que no co...",Muy bueno,Excelente,Excelente,"['El', 'Curry', 'Udon', 'excelente', ',', 'hac...","['el', 'curry', 'udon', 'excelente', ',', 'hac...","['curry', 'udon', 'excelente', ',', 'hacía', '...","['curry', 'udon', 'excelente', 'hacía', 'comía..."
2,-amigos-del-mar-608,06/01/2015,La relación calidad-precio fue mala (muy caro ...,Muy bueno,Regular,Muy bueno,"['La', 'relación', 'calidad-precio', 'fue', 'm...","['la', 'relación', 'calidad-precio', 'fue', 'm...","['relación', 'calidad-precio', 'mala', '(', 'c...","['relación', 'calidad-precio', 'mala', 'caro',..."


In [25]:
def valoracion_a_numeros(valor):
    if valor == 'Excelente':
        return 4
    if valor == 'Muy bueno':
        return 3
    if valor == 'Bueno':
        return 2
    if valor == 'Regular':
        return 1

columnas_valoracion = ['comida','servicio','ambiente']
for item in columnas_valoracion:
    df_reviews[item] = df_reviews[item].apply(valoracion_a_numeros)

print ('valores unicos en COMIDA',df_reviews.comida.unique())
print ('valores unicos en SERVICIO',df_reviews.servicio.unique())
print ('valores unicos en AMBIENTE',df_reviews.ambiente.unique())


valores unicos en COMIDA [4 3 2 1]
valores unicos en SERVICIO [4 1 3 2]
valores unicos en AMBIENTE [4 3 1 2]


In [27]:
mask_negative = (df_reviews["comida"] < 3) & (df_reviews["servicio"] < 3) & (df_reviews["ambiente"] < 3)
mask_positive = (df_reviews["comida"] > 2) & (df_reviews["servicio"] > 2 ) & (df_reviews["ambiente"] >2 )
# mask_com = df_reviews['comida'] = 1
# mask_serv = df_reviews['servicio'] = 1
# mask_amb = df_reviews['ambiente'] = 1
# df_reviews_neg = df_reviews[mask_com] & df_reviews[mask_serv] & df_reviews[mask_amb]

print ('reviews originales',df_reviews.fecha.count())

df_negative = df_reviews[mask_negative]
print ('reviews segmentadas negativas',df_negative.fecha.count())

df_positive = df_reviews[mask_positive]
print ('reviews segmentadas positivas',df_positive.fecha.count())

reviews originales 28286
reviews segmentadas negativas 7419
reviews segmentadas positivas 10802


In [72]:
corpus_positivo = df_positive.nopoint
corpus_negativo = df_negative.nopoint


# Vectorizamos el texto con TfidfVectorizer
vectorizer = TfidfVectorizer(max_df=0.5, max_features=100000,
                             min_df=2, use_idf=True) #le saque el stopwords, porque ya lo hicimos antes

X_positivo = vectorizer.fit_transform(corpus_positivo)
X_negativo = vectorizer.fit_transform(corpus_negativo)

print("n_samples: %d, n_features: %d POSITIVO" % X_positivo.shape)
print("n_samples: %d, n_features: %d NEGATIVO" % X_negativo.shape)

n_samples: 10802, n_features: 8936 POSITIVO
n_samples: 7419, n_features: 9215 NEGATIVO


In [67]:
dist_positivo = nltk.FreqDist(corpus_positivo)
dist_negativo = nltk.FreqDist(corpus_negativo)

In [51]:
dist_negativo

FreqDist({'[]': 4, "['hoy', '25', 'mayo', 'hecho', 'pedido', 'ayer', 'ser', 'entregado', 'hoy', '13', 'registrado', 'hora', 'dirección', 'teléfono', 'siendo', '14', 'dejado', 'clavado', 'entrega', 'nunca', 'llegó', 'nadie', 'responde', 'ninguno', 'teléfonos', 'seriedad', 'proceder', 'requiere', 'mayor', 'comentario']": 3, "['pagina', 'da', 'opción', 'encargar', 'pedido', 'dia', 'siguiente', 'fecha', 'determinada', 'realice', 'manera', 'pedido', 'jamas', 'llego', 'llame', 'restaurant', 'dijeron', 'trabajan', 'así', 'pedido', 'dia', 'pagina', 'especifica', 'pedidos', 'pueden', 'reservar', 'dias', 'antelación', 'sumado', 'atención', 'pésima', 'sumamente', 'irrespetuosos', 'vergüenza']": 3, "['precio', 'platos', 'vinsanto', 'excesivamente', 'oneroso', 'calidad', 'mismos', 'caso', 'pastas', 'salsa', 'económica', 'valor', '$', '68', 'parece', 'despropósito', 'si', 'luego', 'puede', 'descubrir', 'dónde', 'ubicada', 'salsa', 'plato', 'pastas', 'elegido', 'atención', 'pésima', 'mozos', 'saben',

In [31]:
dist_positivo

FreqDist({"['excelente']": 22, "['recomendable']": 6, "['rico']": 6, "['excelente', 'lugar']": 5, "['bueno']": 5, "['ir', 'restaurant', 'comida', 'húngara', 'comer', 'suprema', 'vayas']": 4, "['bien']": 4, "['hola', 'anoche', 'amigos', 'ambiente', 'bueno', 'bien', 'decorado', 'acogedor', 'mesas', 'amplias', 'comida', 'cubrió', 'expectativas', 'excelente', 'presentación', 'platos', 'bien', 'atendido', 'camarera', 'margarita', 'sugerencia', 'detalle', 'considerar', 'falta', 'mantel', 'mesas', 'recomiendo', 'volveremos', '.-']": 4, "['comida', 'unidos', 'bueno', 'porciones', 'pequeno', 'salsas', 'bien', 'necisito', 'dos', 'sandwiches', 'pollo', 'frito', 'si', 'quires', 'satisfecho']": 4, "['riquisimo']": 3, ...})

In [74]:
svd = TruncatedSVD(100,random_state = 1) 
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)
X_positivo = lsa.fit_transform(X_positivo)
X_negativo = lsa.fit_transform(X_negativo)

In [81]:
#tratamiento DF positivo
k_value = 3
from sklearn.cluster import KMeans
km = KMeans(n_clusters=k_value, max_iter=100, n_init=10, random_state = 10)
km.fit(X_positivo)

# Hacemos la transformación inversa para ver el peso de los features originales (10000 dim)
original_space_centroids = svd.inverse_transform(km.cluster_centers_)

# La función argsort nos da los índices ordenados
order_centroids = original_space_centroids.argsort()[:, ::-1]

# Imprimimos los términos más "pesados" (frecuentes) para cada categoría
# Con los índices ordenados, llamamos a la descripción de las palabras que construyó el Vectorizer()
terms = vectorizer.get_feature_names()

print("Ranking de términos por tópico - reviews positivos:")

for i in range(k_value):
    print(end='')
    for ind in order_centroids[i, :15]:
       
        print(' %s' % terms[ind], end='')
    print()

# Nos quedamos con los índices ordenados más importantes en cada tópico

Ranking de términos por tópico - reviews positivos:
 bien mas pedimos si dos plato platos servicio comer pizza carne nunca papas mesa lugar
 mala comida atención atencion lugar mas caro mal recomiendo pedimos nunca vuelvo calidad pizza dos
 buena lugar comida bien pizza atención precios precio comer calidad ambiente rica atencion ir si


In [82]:
#tratamiento DF negativo
k_value = 3
from sklearn.cluster import KMeans
km = KMeans(n_clusters=k_value, max_iter=100, n_init=10, random_state= 3)
km.fit(X_negativo)

# Hacemos la transformación inversa para ver el peso de los features originales (10000 dim)
original_space_centroids = svd.inverse_transform(km.cluster_centers_)

# La función argsort nos da los índices ordenados
order_centroids = original_space_centroids.argsort()[:, ::-1]

# Imprimimos los términos más "pesados" (frecuentes) para cada categoría
# Con los índices ordenados, llamamos a la descripción de las palabras que construyó el Vectorizer()
terms = vectorizer.get_feature_names()

print("Ranking de términos por tópico - reviews negativos:")

for i in range(k_value):
    print(end='')
    for ind in order_centroids[i, :15]:
        print(' %s' % terms[ind], end='')
    print()

# Nos quedamos con los índices ordenados más importantes en cada tópico

Ranking de términos por tópico - reviews negativos:
 lugar bien buena pizza calidad atención comer precio si precios platos caro ambiente ir bueno
 pedimos mas dos mesa nunca plato mal papas trajeron si mozo vino solo minutos desastre
 comida mala lugar atención buena precios servicio ambiente calidad atencion bien si caro mal mas
